In [1]:
import numpy as np
import csv
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
#from tensorflow.examples.tutorials.mnist import input_data
from torch.autograd import Variable
import sklearn.metrics as metrics
import pickle
import os
import numpy.linalg as la
import random
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable


In [19]:
transform = transforms.Compose(
    [transforms.ToTensor()])
train_set = dset.MNIST('/home/daiict/Desktop/udit/C-GAN./data' ,train=True, download= True,
                       transform = transform)
test_set = dset.MNIST('/home/daiict/Desktop/udit/C-GAN./data' ,train=False, download=True,
                       transform = transform)

In [20]:
batch_size = 128

In [21]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,shuffle=True)
print len(train_loader), len(test_loader)

469 79


In [49]:
ip_dim = 28*28
z_dim = 100
comb_dim = ip_dim + z_dim

#### Implementing BIGan

#### Encoder

In [50]:
class encoder(nn.Module):
    def __init__(self):
        super(encoder,self).__init__()
        self.l1 = nn.Linear(ip_dim,500)
        self.l2 = nn.Linear(500,500)
        self.l3 = nn.Linear(500,z_dim)
        
    def forward(self,x):
        x = F.leaky_relu(self.l1(x))
        x = F.leaky_relu(self.l2(x))
        x = F.sigmoid(self.l3(x))
        
        return x

### Decoder

In [51]:
class decoder(nn.Module):
    def __init__(self):
        super(decoder,self).__init__()
        self.l1 = nn.Linear(z_dim,500)
        self.l2 = nn.Linear(500,500)
        self.l3 = nn.Linear(500,ip_dim)
        
    def forward(self,x):
        x = F.leaky_relu(self.l1(x))
        x = F.leaky_relu(self.l2(x))
        x = F.sigmoid(self.l3(x))
        
        return x

###  Discriminator

In [52]:
class disc(nn.Module):
    def __init__(self):
        super(disc,self).__init__()
        self.l1 = nn.Linear(comb_dim,500)
        self.l2 = nn.Linear(500,500)
        self.l3 = nn.Linear(500,500)
        self.l4 = nn.Linear(500,1)
        
    def forward(self,x):
        x = F.leaky_relu(self.l1(x))
        x = F.leaky_relu(self.l2(x))
        x = F.sigmoid(self.l3(x))
        
        return x

In [53]:
G = decoder()
E = encoder()
D = disc()

In [54]:
def clear_grad():
    G.zero_grad()
    E.zero_grad()
    D.zero_grad()

In [55]:
E_solver = optim.Adam(E.parameters())
G_solver = optim.Adam(G.parameters())
D_solver = optim.Adam(D.parameters())

In [56]:
CUDA_LAUNCH_BLOCKING=1

In [65]:
epochs = 1
for ep in range(epochs):
    for i,(x,label) in enumerate(train_loader):
        
        x = Variable(x)
        x = x.view(batch_size,28*28)
    
        zhat = E(x)
        z = Variable(torch.rand(zhat.size()))
        
        xhat = G(z)
        v1 = torch.cat([x,zhat],1)
        
        target = Variable(torch.cat([xhat,z],1).data,requires_grad = False)
        
        loss = nn.BCELoss()
        cost1 = loss(v1,target)
        cost1.backward()
        
        E_solver.step()
        D_solver.step()
        
        clear_grad()
        
        zhat = E(x)
        z = Variable(torch.rand(zhat.size()))
        xhat = G(z)
        target = Variable(torch.cat([x,zhat],1).data,requires_grad = False)
        v1 = torch.cat([xhat,z],1)
        cost2 = nn.BCELoss()(v1,target)
        cost2.backward()
        
        G_solver.step()
        D_solver.step()
        
        clear_grad()
        
        if(ep%15==0):
            print(cost1.data[0],cost2.data[0])
            
        
        
        
        

(10.337264060974121, 0.7277527451515198)
(10.145585060119629, 0.7138038873672485)
(9.789278984069824, 0.691308856010437)
(9.21243667602539, 0.658344030380249)
(8.350910186767578, 0.6090901494026184)
(7.082808494567871, 0.5496007204055786)
(5.69473123550415, 0.4792934060096741)
(3.9030611515045166, 0.43395328521728516)
(2.618997573852539, 0.3910555839538574)
(1.819320797920227, 0.38919198513031006)
(1.4957300424575806, 0.40578693151474)
(1.4750151634216309, 0.3741311728954315)
(1.5595526695251465, 0.37664058804512024)
(1.6713974475860596, 0.3798792362213135)
(1.827377438545227, 0.3661670982837677)
(2.02563738822937, 0.3665293753147125)
(2.203314781188965, 0.36161327362060547)
(2.2350850105285645, 0.36311689019203186)
(2.295563220977783, 0.36514702439308167)
(2.4512085914611816, 0.3619905412197113)
(2.4757635593414307, 0.3582412600517273)
(2.4274418354034424, 0.36339032649993896)
(2.385197401046753, 0.35691359639167786)
(2.419956684112549, 0.3467840850353241)
(2.2722628116607666, 0.35141

KeyboardInterrupt: 